In [ ]:
import pandas as pd
import numpy as np
import math
import random
from tqdm.auto import tqdm

In [ ]:
train_df = pd.read_csv("./data/train.csv")
train_df.head()

,traffic(t-10),traffic(t-9),traffic(t-8),traffic(t-7),traffic(t-6),traffic(t-5),traffic(t-4),traffic(t-3),traffic(t-2),traffic(t-1),traffic(t),type
0,3.0,2.0,5.0,6.0,6.0,4.0,6.0,6.0,3.0,12.0,6.0,0
1,2.0,5.0,6.0,6.0,4.0,6.0,6.0,3.0,12.0,6.0,6.0,0
2,5.0,6.0,6.0,4.0,6.0,6.0,3.0,12.0,6.0,6.0,4.0,0
3,6.0,6.0,4.0,6.0,6.0,3.0,12.0,6.0,6.0,4.0,6.0,0
4,6.0,4.0,6.0,6.0,3.0,12.0,6.0,6.0,4.0,6.0,17.0,0


In [ ]:
from itertools import combinations
combi_list = sorted(combinations(np.arange(11),2))
for combin in list(combi_list):
    i,j = list(combin)[0],list(combin)[1]
    train_df[f'new_feature{i}_{j}'] = train_df.iloc[:,i]/train_df.iloc[:,j]

In [ ]:
train_df.shape

(12645, 67)

# Split and scale train data into stratified 5 folds

In [ ]:
X,y = train_df.drop('type',axis = 1).to_numpy(),train_df['type']

In [ ]:
X_scaled = (X-X.mean())/X.std()

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5,random_state=38,shuffle = True)
skf.get_n_splits(X_scaled, y)

5

In [ ]:
global fold_dict
fold_dict = dict()

for i, (train_index, test_index) in enumerate(skf.split(X_scaled,y)):
    fold_dict[f'fold{i}'] = {'train_indices':train_index, 'test_indices':test_index}

In [ ]:
fold_dict.keys()

dict_keys(['fold0', 'fold1', 'fold2', 'fold3', 'fold4'])

In [ ]:
from collections import Counter
print(Counter(y[fold_dict['fold0']['test_indices']]))
print(Counter(y[fold_dict['fold1']['test_indices']]))
print(Counter(y[fold_dict['fold2']['test_indices']]))
print(Counter(y[fold_dict['fold3']['test_indices']]))
print(Counter(y[fold_dict['fold4']['test_indices']]))

Counter({0: 1578, 1: 588, 2: 182, 3: 181})
Counter({0: 1577, 1: 589, 2: 182, 3: 181})
Counter({0: 1577, 1: 589, 3: 182, 2: 181})
Counter({0: 1577, 1: 589, 3: 182, 2: 181})
Counter({0: 1577, 1: 588, 2: 182, 3: 182})


# Model

In [ ]:
from tqdm.auto import tqdm

In [ ]:
class rbfSVMClassifier:
    def __init__(self,n_iters=5, lr = 0.0001, random_seed=3, sigma=1):
        self.n_iters = n_iters # 몇 회 반복하여 적절한 값을 찾을지 정하는 파라미터
        self.lr = lr  # 학습률과 관련된 파라미터 
        self.sigma = sigma
        self.random_seed = random_seed
        np.random.seed(self.random_seed)

    def rbf_kernel(self,x,z):
        numer = -(x-z).dot(x-z)
        denom = 2*self.sigma**2
        return math.exp(numer/denom) 
    
    def sign(self,scalar):
        if scalar >= 0:
            return 1
        else:
            return 0
    
    def update_weight(self,x_i,y_i,i):
        summ = 0
        if i == 0:
            b_next = self.lr*(y_i-self.sign(0))
            self.weight = np.append(self.weight,b_next)
            return self.weight
        else:
            for j in range(i):
                summ += self.weight[j]*self.rbf_kernel(self.train_x[j],x_i)
            b_next = self.lr*(y_i-self.sign(summ))
            self.weight =np.append(self.weight,b_next)
            return self.weight
    
    def fit(self, x_, y_orig):
        """
        본 함수는 x, y를 활용하여 훈련하는 과정을 코딩하는 부분입니다.
        아래 reference 사이트의 gradient 계산 부분을 참고하세요.
        reference: https://towardsdatascience.com/support-vector-machine-introduction-to-machine-learning-algorithms-934a444fca47
        아래 총 6개의 None 부분을 채우시면 됩니다.

        """
        n_samples, n_features = x_.shape
        # hint: y값을 SVM 계산에 활용해주기 위하여 0에 해당하는 y값들을 -1로 변환
        y_ =  np.array(y_orig)#numpy array의 y를 y_에 담아줍니다       
        y_[y_ == 0] = -1#0에 해당하는 y_값들을 -1로 변환합니다.
        
        self.weight = np.array([])
        
        n_label = np.sum((y_==1)*1)
        
        for _ in tqdm(range(self.n_iters),leave =False, desc = f"training for {self.n_iters} iterations"):
            
            ###### Oversampling #######
            non_label_indices = np.where(y_ == -1)[0]
            label_indices = np.where(y_ != -1)[0]
            more_sample_n = len(non_label_indices)-len(label_indices)
            n_iter = _
            if more_sample_n <0:
                more_sample_indices = np.random.choice(non_label_indices,size = -more_sample_n,replace = False)
            else:
                more_sample_indices = np.random.choice(label_indices,size = more_sample_n,replace = True)
            x = np.vstack([x_,x_[more_sample_indices,:]])
            y = np.concatenate([y_,y_[more_sample_indices]])
            self.train_x = x
            self.train_y = y 
            n_samples_oversampled = len(y)
            assert Counter(y)[list(Counter(y).keys())[0]] == Counter(y)[list(Counter(y).keys())[1]]
            assert x.shape[0] == y.shape[0]
            for i in tqdm(range(n_samples_oversampled),leave = False, desc = "updating through all train data"):
                x_i = x[i]
                y_i = y[i]
                #### Update considering gradient
                self.weight = self.update_weight(x_i,y_i,i)
                
        return self.weight
    
    def predict_(self,x):#predict for single instance
        summ = 0
        for idx, x_ in enumerate(self.train_x):
            summ += self.weight[idx]*self.rbf_kernel(x,x_)
        return self.sign(summ)
    
    def predict(self, x):
        n_test_data = x.shape[0]
        preds = []
        for i in range(n_test_data):
            pred = self.predict_(x[i,:])
            preds.append(pred)
        preds = np.array(preds)
        return preds#예측 값을 담은 배열인 approx를 반환합니다
    
    def sigmoid(self,z):
        return 1./(1.+np.exp(-z))
    
    def get_prob_(self,x_i):
        summ = 0
        for idx, x_ in enumerte(self.train_x):
            summ += self.weight[idx]*self.rbf_kernel(x_i,x_)
        return self.sigmoid(summ)
    
    def get_prob(self,X):
        n_test_data = x.shape[0]
        probs = []
        for i in range(n_test_data):
            prob = self.get_prob_(x[i,:])
            probs.append(prob)
        probs = np.array(probs)
        return probs


    def get_accuracy(self, y_true, y_pred):
        """
            y_true, y_pred가 들어왔을 때, 정확도를 계산하는 함수.
            sklearn의 accuracy_score 사용 불가능 / sklearn의 accuracy_score 함수를 구현한다고 생각하면 됩니다.
            넘파이만을 활용하여 정확도 계산 함수를 작성하세요.
        """
        acc = np.sum(y_true == y_pred)/len(y_pred)#예측하고자 하는 전체 데이터 수 개수(len(y_pred)) 중 예측 값과 실제 값이 일치하는 데이터 개수(np.sum(y_true == y_pred))의 비율로 정확도를 계산합니다
        return acc #정확도를 담은 acc를 반환합니다

# Utils

In [ ]:
def train_test_binary_model(model,label):
    avg_train_acc = []
    avg_test_acc = []
    for fold in tqdm(fold_dict.values(),leave=False,desc = "5 folds..."):
        train_X,train_y = X_scaled[fold['train_indices']],y[fold['train_indices']]
        test_X,test_y = X_scaled[fold['test_indices']],y[fold['test_indices']]
        use_train_y,use_test_y = (train_y==label)*1,(test_y==label)*1
        model.fit(train_X,use_train_y)
        train_preds = model.predict(train_X)
        train_acc = np.mean(train_preds == use_train_y)
        test_preds = model.predict(test_X)
        test_acc = np.mean(test_preds == use_test_y)
        avg_train_acc.append(train_acc)
        avg_test_acc.append(test_acc)
    return np.mean(np.array(avg_train_acc)),np.mean(np.array(avg_test_acc))

In [ ]:
test_df = pd.read_csv("./data/test.csv")
from itertools import combinations
combi_list = sorted(combinations(np.arange(11),2))
for combin in list(combi_list):
    i,j = list(combin)[0],list(combin)[1]
    test_df[f'new_feature{i}_{j}'] = test_df.iloc[:,i]/test_df.iloc[:,j]
test_df.head()

,traffic(t-10),traffic(t-9),traffic(t-8),traffic(t-7),traffic(t-6),traffic(t-5),traffic(t-4),traffic(t-3),traffic(t-2),traffic(t-1),...,new_feature6_7,new_feature6_8,new_feature6_9,new_feature6_10,new_feature7_8,new_feature7_9,new_feature7_10,new_feature8_9,new_feature8_10,new_feature9_10
0,61.0,26.0,29.0,37.0,20.0,32.0,25.0,25.0,16.0,18.0,...,1.000000,1.562500,1.388889,1.136364,1.562500,1.388889,1.136364,0.888889,0.727273,0.818182
1,2.0,10.0,11.0,28.0,21.0,18.0,5.0,6.0,10.0,3.0,...,0.833333,0.500000,1.666667,0.714286,0.600000,2.000000,0.857143,3.333333,1.428571,0.428571
2,14.0,1.0,14.0,14.0,14.0,7.0,20.0,14.0,9.0,7.0,...,1.428571,2.222222,2.857143,1.000000,1.555556,2.000000,0.700000,1.285714,0.450000,0.350000
3,20.0,7.0,25.0,15.0,21.0,13.0,21.0,14.0,8.0,20.0,...,1.500000,2.625000,1.050000,1.500000,1.750000,0.700000,1.000000,0.400000,0.571429,1.428571
4,155.0,71.0,10.0,23.0,480.0,617.0,58.0,1019.0,166.0,2.0,...,0.056919,0.349398,29.000000,29.000000,6.138554,509.500000,509.500000,83.000000,83.000000,1.000000


In [ ]:
import torch
X_test = test_df.to_numpy()
X_test_scaled = (X_test-X_test.mean())/X_test.std()

In [ ]:
import copy

lrs = [0.1,0.01,0.005,0.0005]
C = [0.1,1]
best_param0 = dict()
best_acc_0 = 0
best_train_acc = 0
save_models = dict()
for C_ in C:
    for lr in lrs:
        svm0 = rbfSVMClassifier(sigma=C_,lr=lr)
        train_acc,test_acc = train_test_binary_model(svm0,1)
        print(f"(lr:{lr}, C:{C_} label:0) acc: {round(train_acc,4)} val acc:{round(test_acc,3)}")
        if train_acc > best_train_acc and test_acc>=best_acc_0:
            print(">>best model updated\n")
            best_acc_0 = test_acc
            best_param0['lr'] = lr
            best_param0['lambda']=C_
            best_svm0 = copy.deepcopy(svm0)
            best_train_acc = train_acc
        if test_acc > 0.9:
            save_models[test_acc] = {'train_acc':train_acc,'model':copy.deepcopy(svm0)}
            test_preds = svm0.predict(X_test_scaled)
            torch.save({'train_acc':train_acc,'model':copy.deepcopy(svm0),'test_preds':test_preds},f"./rbf/label1_{str(test_acc)[:4]}.pt")

5 folds...:   0%|          | 0/5 [00:00<?, ?it/s]

training for 5 iterations:   0%|          | 0/5 [00:00<?, ?it/s]

updating through all train data:   0%|          | 0/15522 [00:00<?, ?it/s]

updating through all train data:   0%|          | 0/15522 [00:00<?, ?it/s]

updating through all train data:   0%|          | 0/15522 [00:00<?, ?it/s]

...

updating through all train data:   0%|          | 0/15524 [00:00<?, ?it/s]

updating through all train data:   0%|          | 0/15524 [00:00<?, ?it/s]

updating through all train data:   0%|          | 0/15524 [00:00<?, ?it/s]

updating through all train data:   0%|          | 0/15524 [00:00<?, ?it/s]

updating through all train data:   0%|          | 0/15524 [00:00<?, ?it/s]

In [ ]:
print("best_train_acc:",best_train_acc,"best test acc:",best_acc_0)
best_param0

In [ ]:
import torch

save_name = f"./weights/rbf_oversampling_label1_{str(best_acc_0)[:4]}.pt"
best_param0['train_acc'] = best_train_acc
best_param0['test_acc'] = best_acc_0
best_param0['best_model'] = best_svm0
torch.save(best_param0,save_name)

In [ ]:
test_df = pd.read_csv("./data/test.csv")
test_df.head()